# Análise de Sentimento simples com a blibioteca TextBlob e Spark Streaming


##  Leitura por meio de socket

###  Seção de Conexão

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = (
    SparkSession
    .builder
    .appName("StructureNetworkWordCount")
    .getOrCreate()
    )

## Bibliotecas para a analise e contagem

In [2]:
from textblob import TextBlob
from googletrans import Translator
from unidecode import unidecode

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, FloatType


In [3]:
#Dataframe que irá ler cada linha recebida pelo localhost e definição da fonte de dados
twitters = \
    (
        spark
        .readStream
        .format("socket")
        .option("host","localhost")
        .option("port", 9995)
        .load()
    )

## Função para traduzir para o inglês

In [4]:
def translate_udf(col):
    translate_obj = Translator().translate(col)
    return translate_obj.text

## Função para traduzir para realizar a análise de sentimento


In [5]:
def sentiment_udf(col):
    sentiment_text = TextBlob(col)
    return sentiment_text.polarity()

## Difinição das Funções como User-Defined-Function

In [7]:
unicode_udf_string = udf(lambda z: unidecode(z), StringType())
group_by_sentiment = udf(lambda x: 'negativo' if x < -0.1 else 'positivo' if x > 0.1 else 'neutro', StringType()) #classifica o sentimento
translate_udf_string = udf(translate_udf, StringType()) #define função de tradução
sentiment_udf_float = udf(sentiment_udf, FloatType()) #define função de contagem